# Desafio Sprint 3

## Etapa 1: ambiente

Nesta etapa devemos certificar que as bibliotecas **pandas** e **matplotlib** estão instaladas no ambiente atual.
Podemos importar ambas as bibliotecas com o comando:
```bash
python.exe -m pip install matplotlib pandas
```

Ao longo do projeto, também serão necessárias algumas bibliotecas adicionais.
```bash
python.exe -m pip install tabulate
```

Portanto recomenda-se instalar os pacotes necessários lendo o arquivo [requirements.txt](requirements.txt):
```bash
# Rodar comando na pasta do desafio
python.exe -m pip install -r requirements.txt
```

Podemos conferir a lista de pacotes instalados usando o comando:
```bash
python.exe -m pip freeze
```

## Etapa 2: desenvolvimento

Funcões utilitárias:

In [1]:
import pandas as pd # biblioteca de tratamento de dados
from IPython.display import display, Markdown # Exibição dos dados em tabelas markdown
from random import randint  # Funcao para gerar numeros aleatorios

#Nome das colunas do CSV como constantes
APP =                  "App"
CATEGORY =        "Category"
RATING =            "Rating"
REVIEWS =          "Reviews"
SIZE =               "Size"
INSTALLS =        "Installs"
TYPE =                "Type"
PRICE =              "Price"
AGE_RANGE = "Content Rating"
GENRE =            "Genres"
L_UPDATE =    "Last Updated"
C_VERSION =    "Current Ver"
A_VERSION =    "Android Ver"

# Funcao para exibir dataframes como tabelas markdown no notebook
def display_df_table(table: pd.DataFrame, show_index=True) -> None:
    display(Markdown(table.to_markdown(index=show_index)))

### 1. Leitura e Tratamento dos Dados de [googleplaystore.csv](./googleplaystore.csv)

Como primeiro passo devemos importar os dados. 

In [2]:
# Importando arquivo csv e convertendo para objeto dataframe
raw_dataframe = pd.read_csv("googleplaystore.csv") # Caminho relativo ao path deste notebook

# Mostrando primeiro registro do csv
display_df_table(raw_dataframe.head(1), False)

| App                                            | Category       |   Rating |   Reviews | Size   | Installs   | Type   |   Price | Content Rating   | Genres       | Last Updated    | Current Ver   | Android Ver   |
|:-----------------------------------------------|:---------------|---------:|----------:|:-------|:-----------|:-------|--------:|:-----------------|:-------------|:----------------|:--------------|:--------------|
| Photo Editor & Candy Camera & Grid & ScrapBook | ART_AND_DESIGN |      4.1 |       159 | 19M    | 10,000+    | Free   |       0 | Everyone         | Art & Design | January 7, 2018 | 1.0.0         | 4.0.3 and up  |

Em seguida podemos remover todos os registros duplicados

In [3]:
# Removendo duplicatas
distinct_dataframe = raw_dataframe.drop_duplicates()
removed_registers = (len(raw_dataframe) - len(distinct_dataframe))

print(f"Número de Registros Bruto: {len(raw_dataframe):>14}")
print(f"Número de Registros Distintos: {len(distinct_dataframe):>10}")
print(f"Número de Registros Removidos: {removed_registers:>10}")

Número de Registros Bruto:          10841
Número de Registros Distintos:      10358
Número de Registros Removidos:        483


Podemos agora analisar o formato dos dados de cada coluna e tratar/filtrar de forma mais profunda

#### 1. Coluna "App"

Vamos observar os dados dessa coluna:

In [4]:
# Exibindo 10 registros variados da coluna "App"
display_df_table(distinct_dataframe[::randint(700, 1000)][APP].head(10))
# Este bloco pode ser executado diversas vezes para ter maior noção dos dados

|      | App                                            |
|-----:|:-----------------------------------------------|
|    0 | Photo Editor & Candy Camera & Grid & ScrapBook |
|  827 | Lerni. Learn languages.                        |
| 1661 | Temple Run 2                                   |
| 2462 | WAH 247                                        |
| 3324 | TunnelBear VPN                                 |
| 4083 | Amazon Kindle                                  |
| 4817 | War Z 2                                        |
| 5546 | Stack It AR                                    |
| 6275 | BI APP                                         |
| 7010 | Pixyfy: coloring by number coloring book       |

Podemos notar que nesta coluna praticamente tem um pouco de tudo. 

Como o nome do aplicativo é uma informação muito importante para sua identificação, vamos apenas checar se a coluna possuí algum valor nulo.

In [158]:
# Conta quantos valores em "App" sao nulos
app_missing_count = distinct_dataframe[APP].isnull().sum()
null_app_percent = (app_missing_count / len(distinct_dataframe)) * 100


print(f"Valores nulos na coluna {APP}: {app_missing_count:>22}")
print(f"Porcentagem de valores nulos na coluna {APP}: {null_app_percent:>6.2f}%")


Valores nulos na coluna App:                      0
Porcentagem de valores nulos na coluna App:   0.00%


O arquivo não contém nenhum valor nulo na coluna "App", portanto não precisamos realizar nenhum tratamento.

#### 2. Coluna "Category"

Vamos observar os dados dessa coluna:

In [6]:
# Exibindo 10 registros variados da coluna "Category"
display_df_table(distinct_dataframe[::randint(700, 1000)][CATEGORY].head(10))
# Este bloco pode ser executado diversas vezes para ter maior noção dos dados

|      | Category           |
|-----:|:-------------------|
|    0 | ART_AND_DESIGN     |
|  969 | ENTERTAINMENT      |
| 1889 | GAME               |
| 2858 | PHOTOGRAPHY        |
| 3772 | NEWS_AND_MAGAZINES |
| 4635 | FAMILY             |
| 5472 | FAMILY             |
| 6311 | GAME               |
| 7157 | NEWS_AND_MAGAZINES |
| 7995 | GAME               |

Podemos notar que o formato padrão para esta coluna são palavras em letras maíuscula separadas por *underline*.
Vamos filtrar os registros que não seguem esta regra:

In [7]:
# Expressão regular que checa letras maíusculas e underline
category_pattern = r"[A-Z_]"

# Cria um objeto Series com valores True|False que serve como máscara de busca 
category_mask = distinct_dataframe[CATEGORY].str.match(category_pattern)

# Usa máscara criada para filtrar as linhas com "Category" válidas
valid_category_dataframe = distinct_dataframe[category_mask]

removed_registers = (len(distinct_dataframe) - len(valid_category_dataframe))

print(f"Número de Registros Bruto: {len(distinct_dataframe):>14}")
print(f"Número de Registros Válidos: {len(valid_category_dataframe):>12}")
print(f"Número de Registros Removidos: {removed_registers:>10}")

Número de Registros Bruto:          10358
Número de Registros Válidos:        10357
Número de Registros Removidos:          1


#### 3. Coluna "Rating"

Esta coluna representa a avaliação de um aplicativo. Como indicado pelas [políticas de avaliação](https://play.google/intl/pt-BR/comment-posting-policy/) da plataforma, a avaliação de um aplicativo é um valor entre 1 e 5 que representa a média de avaliações dos usuários da ultima versão do aplicativo (campo "Current Ver").

Vamos verificar a quantidade de valores fora do range 1-5 e também quantos valores nulos temos:


In [160]:
out_of_range_rating_dataframe = valid_category_dataframe[
    # Seleciona apenas registros com Rating fora do range 1-5
    (valid_category_dataframe[RATING] < 1) | (valid_category_dataframe[RATING] > 5)
]

rating_missing_count = valid_category_dataframe[RATING].isnull().sum()
null_rating_percent = (rating_missing_count / len(valid_category_dataframe)) * 100

print(f"Valores fora do range válido em {RATING}: {len(out_of_range_rating_dataframe):>14}")
print(f"Valores nulos na coluna {RATING}: {rating_missing_count:>22}")
print(f"Porcentagem de valores nulos na coluna {RATING}: {null_rating_percent:>6.2f}%")

Valores fora do range válido em Rating:              0
Valores nulos na coluna Rating:                   1465
Porcentagem de valores nulos na coluna Rating:  14.15%


Não temos números fora do range estabelecido para serem removidos.

Como a quantidade de valores nulos em "Rating" representa uma parcela considerável dos valores e provavelmente indica que o aplicativo não possui avaliações em quantidade suficiente, é preferível mantê-los para aumentar a quantidade de registros disponíveis para análise de outros fatores.

#### 4. Coluna "Reviews

Seguindo a [políticas de avaliação](https://play.google/intl/pt-BR/comment-posting-policy/) da plataforma, podemos assumir que este campos representa a quantidade total de avaliações que um aplicativo possui

Vamos verificar valores inválidos, podemos considerar inválido qualquer valor fora do conjunto $\mathbb{N}_0$:

In [194]:
# Quantidade de valores nulos na coluna "Reviews"
reviews_missing_count = valid_category_dataframe[REVIEWS].isnull().sum()

# Mascara que valida apenas valores inteiros
reviews_int_mask = valid_category_dataframe[REVIEWS].apply(lambda n: int(n) == float(n))

# Mascara que valida apenas valores positivos
reviews_unsigned_mask = valid_category_dataframe[REVIEWS].apply(lambda n: int(n) >= 0)

# Usa as mascaras criadas para retornar apenas os valores invalidos
reviews_invalid_count = valid_category_dataframe[REVIEWS][~reviews_int_mask | ~reviews_unsigned_mask].sum()

reviews_remove_count = reviews_missing_count + reviews_invalid_count
remove_reviews_percent = (reviews_invalid_count / len(valid_category_dataframe)) * 100


print(f"Valores inválidos na coluna {REVIEWS}: {reviews_remove_count:>22}")
print(f"Porcentagem de valores inválidos na coluna {REVIEWS}: {remove_reviews_percent:>6.2f}%")

Valores inválidos na coluna Reviews:                      0
Porcentagem de valores inválidos na coluna Reviews:   0.00%


Não temos valores inválidos nesta coluna

#### 5. Coluna "Size"